# 모두를 위한 딥러닝 Lab4_2
### Loading data from file

In [2]:
import tensorflow as tf
import numpy as np

In [4]:
xy = np.loadtxt('data-01-test-score.csv',delimiter=',',dtype=np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]

In [9]:
X = tf.placeholder(tf.float32, shape=[None,3])
Y = tf.placeholder(tf.float32, shape=[None,1])

W = tf.Variable(tf.random_normal([3,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [10]:
hypothesis = tf.matmul(X,W) + b
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [12]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train],
                                   feed_dict={ X : x_data, Y : y_data})
    if step % 100 == 0:
        print(step, "Cost:",cost_val,"\nPrediction:\n",hy_val)

0 Cost: 7100.7227 
Prediction:
 [[82.82099 ]
 [85.75617 ]
 [91.925545]
 [98.1094  ]
 [63.277367]
 [41.25066 ]
 [73.78854 ]
 [55.762432]
 [74.13442 ]
 [62.52845 ]
 [69.030464]
 [60.987488]
 [96.97319 ]
 [82.45248 ]
 [71.788246]
 [86.30339 ]
 [76.67336 ]
 [90.799164]
 [94.889305]
 [85.86835 ]
 [85.551476]
 [79.15606 ]
 [83.29911 ]
 [88.132225]
 [92.16181 ]]
100 Cost: 66.282616 
Prediction:
 [[161.28198]
 [180.66728]
 [185.11661]
 [199.75935]
 [135.68353]
 [ 94.9473 ]
 [149.94102]
 [112.2887 ]
 [163.16948]
 [145.39973]
 [142.41875]
 [133.67876]
 [193.10173]
 [162.17888]
 [148.39911]
 [182.70998]
 [152.5995 ]
 [182.14825]
 [186.41536]
 [167.72514]
 [174.88351]
 [168.31046]
 [168.37366]
 [166.88576]
 [190.35646]]
200 Cost: 61.184444 
Prediction:
 [[160.92993]
 [180.82678]
 [184.96115]
 [199.74089]
 [135.8255 ]
 [ 95.35283]
 [149.98808]
 [112.39141]
 [163.63802]
 [146.21928]
 [142.48282]
 [134.05688]
 [192.797  ]
 [161.74918]
 [148.55006]
 [182.96176]
 [152.18796]
 [182.16493]
 [186.01114]
 

In [16]:
print(sess.run(hypothesis,feed_dict={X:[[100,70,101]]}))

[[170.241]]


### Queue Runners

In [17]:
import tensorflow as tf

##### 1. 파일 여러개를 filename_queue에 넣는다

In [20]:
# 지금은 한개만 있는 경우
filename_queue = tf.train.string_input_producer(['data-01-test-score.csv'],shuffle=False, name='filename_queue')

##### 2. filename_queue에서 한개 씩 읽는다

In [25]:
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

##### 3. file을 decode한다

In [111]:
record_defaults = [[0.],[0.],[0.],[0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [112]:
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1],xy[-1:]],batch_size=100)

In [113]:
X = tf.placeholder(tf.float32,shape=[None,3])
Y = tf.placeholder(tf.float32,shape=[None,1])

W = tf.Variable(tf.random_normal([3,1]),name='weight')
b = tf.Variable(tf.random_normal([1]),name='bias')

In [114]:
hypothesis = tf.matmul(X,W) + b
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [115]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [116]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [117]:
# filename queue
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [118]:
for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch,train_y_batch])
    cost_val,hy_val, _ = sess.run([cost,hypothesis,train],
                                  feed_dict={X : x_batch, Y : y_batch})
    if step % 100 == 0:
        print(step,"Cost:",cost_val,"\nPrediction:\n",hy_val)
        
coord.request_stop()
coord.join(threads)

0 Cost: 2891.7825 
Prediction:
 [[101.21301 ]
 [127.97379 ]
 [122.71031 ]
 [121.36108 ]
 [107.647026]
 [132.85403 ]
 [129.0318  ]
 [108.26676 ]
 [132.72266 ]
 [122.71031 ]
 [ 97.47993 ]
 [108.26676 ]
 [129.19977 ]
 [111.80518 ]
 [106.87007 ]
 [119.47878 ]
 [103.45738 ]
 [ 99.97109 ]
 [129.19977 ]
 [120.783455]
 [121.36108 ]
 [ 95.35085 ]
 [ 95.35085 ]
 [108.26676 ]
 [106.983154]
 [107.647026]
 [ 73.9244  ]
 [101.21301 ]
 [ 64.68614 ]
 [132.72266 ]
 [132.72266 ]
 [127.97379 ]
 [111.80518 ]
 [ 73.9244  ]
 [ 95.24575 ]
 [ 64.68614 ]
 [121.36108 ]
 [110.9575  ]
 [ 95.35085 ]
 [ 97.47993 ]
 [127.97379 ]
 [108.26676 ]
 [ 96.88125 ]
 [129.19977 ]
 [106.983154]
 [111.80518 ]
 [120.783455]
 [106.87007 ]
 [113.422386]
 [119.47878 ]
 [107.647026]
 [103.45738 ]
 [132.85403 ]
 [ 99.97109 ]
 [129.0318  ]
 [122.71031 ]
 [132.72266 ]
 [101.21301 ]
 [ 73.9244  ]
 [ 95.24575 ]
 [ 64.68614 ]
 [121.36108 ]
 [110.9575  ]
 [ 95.35085 ]
 [ 97.47993 ]
 [127.97379 ]
 [108.26676 ]
 [ 96.88125 ]
 [129.19977 ]
 [

600 Cost: 29.381357 
Prediction:
 [[162.57281]
 [153.27243]
 [194.64038]
 [150.33705]
 [188.69676]
 [181.91539]
 [197.59401]
 [146.17714]
 [107.57909]
 [144.63284]
 [102.48277]
 [177.4003 ]
 [163.65381]
 [142.26994]
 [143.50981]
 [188.72162]
 [158.14377]
 [146.40884]
 [190.20088]
 [153.7368 ]
 [171.52464]
 [178.76932]
 [158.91928]
 [171.07178]
 [175.89235]
 [162.57281]
 [153.27243]
 [194.64038]
 [150.33705]
 [188.69676]
 [181.91539]
 [197.59401]
 [146.17714]
 [107.57909]
 [144.63284]
 [102.48277]
 [177.4003 ]
 [163.65381]
 [142.26994]
 [143.50981]
 [188.72162]
 [158.14377]
 [146.40884]
 [190.20088]
 [153.7368 ]
 [171.52464]
 [178.76932]
 [158.91928]
 [171.07178]
 [175.89235]
 [162.57281]
 [153.27243]
 [194.64038]
 [150.33705]
 [188.69676]
 [181.91539]
 [197.59401]
 [146.17714]
 [107.57909]
 [144.63284]
 [102.48277]
 [177.4003 ]
 [163.65381]
 [142.26994]
 [143.50981]
 [188.72162]
 [158.14377]
 [146.40884]
 [190.20088]
 [153.7368 ]
 [171.52464]
 [178.76932]
 [158.91928]
 [171.07178]
 [17

1200 Cost: 23.096186 
Prediction:
 [[163.27911]
 [152.72188]
 [194.08177]
 [150.57907]
 [188.18307]
 [181.79744]
 [197.78712]
 [145.32002]
 [107.34976]
 [145.48195]
 [104.12096]
 [177.12674]
 [164.03976]
 [142.51767]
 [143.52249]
 [188.25462]
 [157.23952]
 [147.1773 ]
 [190.06102]
 [152.23462]
 [172.93372]
 [178.39676]
 [158.73161]
 [171.83742]
 [175.77666]
 [163.27911]
 [152.72188]
 [194.08177]
 [150.57907]
 [188.18307]
 [181.79744]
 [197.78712]
 [145.32002]
 [107.34976]
 [145.48195]
 [104.12096]
 [177.12674]
 [164.03976]
 [142.51767]
 [143.52249]
 [188.25462]
 [157.23952]
 [147.1773 ]
 [190.06102]
 [152.23462]
 [172.93372]
 [178.39676]
 [158.73161]
 [171.83742]
 [175.77666]
 [163.27911]
 [152.72188]
 [194.08177]
 [150.57907]
 [188.18307]
 [181.79744]
 [197.78712]
 [145.32002]
 [107.34976]
 [145.48195]
 [104.12096]
 [177.12674]
 [164.03976]
 [142.51767]
 [143.52249]
 [188.25462]
 [157.23952]
 [147.1773 ]
 [190.06102]
 [152.23462]
 [172.93372]
 [178.39676]
 [158.73161]
 [171.83742]
 [1

1900 Cost: 17.902567 
Prediction:
 [[163.97816]
 [152.24414]
 [193.52946]
 [150.85193]
 [187.66013]
 [181.69577]
 [197.97975]
 [144.45937]
 [107.08421]
 [146.31654]
 [105.72998]
 [176.81154]
 [164.34268]
 [142.7563 ]
 [143.49878]
 [187.8223 ]
 [156.38701]
 [147.92226]
 [189.8987 ]
 [150.78944]
 [174.32462]
 [178.0673 ]
 [158.58417]
 [172.58589]
 [175.63747]
 [163.97816]
 [152.24414]
 [193.52946]
 [150.85193]
 [187.66013]
 [181.69577]
 [197.97975]
 [144.45937]
 [107.08421]
 [146.31654]
 [105.72998]
 [176.81154]
 [164.34268]
 [142.7563 ]
 [143.49878]
 [187.8223 ]
 [156.38701]
 [147.92226]
 [189.8987 ]
 [150.78944]
 [174.32462]
 [178.0673 ]
 [158.58417]
 [172.58589]
 [175.63747]
 [163.97816]
 [152.24414]
 [193.52946]
 [150.85193]
 [187.66013]
 [181.69577]
 [197.97975]
 [144.45937]
 [107.08421]
 [146.31654]
 [105.72998]
 [176.81154]
 [164.34268]
 [142.7563 ]
 [143.49878]
 [187.8223 ]
 [156.38701]
 [147.92226]
 [189.8987 ]
 [150.78944]
 [174.32462]
 [178.0673 ]
 [158.58417]
 [172.58589]
 [1